In [22]:
import pipeline_funcs as fn
import os

import pandas as pd
pd.options.mode.chained_assignment = None

In [ ]:
root_path = '..'
yesterday = '2024-12-31' #if yesterday is later than snapshot_date, use snapshot_date
last_executed_date = '2024-12-15' #check previous folder date
snapshot_date = '20241231' #date to create table in seaocdm-data-hub.gcdm_snapshots.campaigns_YYYYMMDD

bo_report = f'{root_path}/data/{yesterday}/All Campaign Summary.csv'

campaign_name_mapper = f'{root_path}/lib/campaign_name_mapper.xlsx'
text_content_mapper = f'{root_path}/lib/text_content_mapper.xlsx'

text_folder_path = f'{root_path}/data/{yesterday}/text_extract'

out_path = f'{root_path}/output/{yesterday}'
out_path_data_hub = f'{out_path}/Campaign Table Update - Data Hub.parquet'
out_path_data_ = f'{out_path}/Campaign Table Update - Data .parquet'
out_path_preview = f'{out_path}/Preview.xlsx'
out_path_data_all_models = f'{out_path}/Campaign Table Update - All Models.parquet'

service_cred_path = f'../../../docs/api_key/xxx.json'
client_secret_path = f'../../../docs/api_key/xxx.json'

In [11]:
for folder_path in [text_folder_path, out_path]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [24]:
df = fn.clean_bo_report(file_path=bo_report)

In [25]:
df = fn.model_tag_campaign_name(df=df, mapper_path=campaign_name_mapper)

In [26]:
df = fn.model_tag_extracted_text(
    df=df,
    mapper_path=text_content_mapper,
    text_folder_path=text_folder_path,
    last_executed_date=last_executed_date,
    service_cred_path=service_cred_path
)


Campaign nz/0000202443 not found in creative-edm-text!
Campaign sg/0000202833 not found in creative-edm-text!


In [ ]:
fn.export_results(
    df=df,
    out_path_data_hub=out_path_data_hub,
    out_path_data=out_path_data_,
    out_path_preview=out_path_preview,
    out_path_data_all_models=out_path_data_all_models
)

Next step is optional, please comment out IMMEDIATELY after run

In [ ]:
fn.manual_revision(
    preview=out_path_preview,
    out_path_data_hub=out_path_data_hub,

    out_path_data_=out_path_data_,
    out_path_data_all_models=out_path_data_all_models
)

In [ ]:
fn.update_bq_table(snapshot_date, out_path_data_hub, out_path_data_, out_path_data_all_models, client_secret_path, run_list=[1,2,3,4]) #run_list=[1,2,3,4,5] #Query 5 should be run when updating campaign dashboard